
---
<big><big><big><big><big><big>Sieci neuronowe 2018/19</big></big></big></big></big></big>

---
<big><big><big><big><big>Odległość Wassersteina</big></big></big></big></big>

---

In [ ]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

plt.style.use("fivethirtyeight")

from bokeh.io import gridplot, output_file, show
from bokeh.plotting import figure, output_notebook
from bkcharts import Scatter

In [ ]:
output_notebook()

In [ ]:
sns.set(font_scale=2.0)

Image inclusion
<img src="../nn_figures/" width="100%">

# Autoenkoder z miarą Wassersteina
* tzw. optimal-transport OT jest metodą porównywania rozkładów
  * wiele silnych definicji odległości (różne dywargencje) często nie dają uzytecznych dla nauczania gradientów
  * OT zachowuje się "przyjemniej"
  * w GANach wymaga jednak spełnienia pewnych ograniczeń

* WAE minimalizują koszt _transportu_ $W_c(P_X,P_G)$ dla dowolnej funkcji kosztu $c$
* koszt składa się z
  * błędu rekonstrukcji $c$
  * regularyzatora $D_Z(P_Z,Q_Z)$ karzących za niezgodność rozkładów
    * założonego prioru $P_Z$
    * rozkładu zakodowanych danych $Q_Z=\mathbb{E}_{P_Z}[Q(Z\min X]$
      * dla kwadratowego $c$ i celu GAN, WAE jest zgodny z adwersarialnymi auto-enkoderami

## Problem transportowy
* (wiki) mamy $m$ kopalni i $n$ fabryk wykorzystujących kopalinę
  * niech $$c:\mathbb{R}^2\times\mathbb{R}^2\longrightarrow[0,\infty)$$ jest kosztem przewiezienia kopalin z kopalni do fabryki (tu na płaszczyźnie $\mathbb{R}^2$)
    * produkcję jednej kopalni można przewieźć do jednej fabryki
  * $T:K\longrightarrow F$ jest planem (transportem) z kopalni do fabryk
  * potrzebujemy __optymalnego__ planu, który minimalizuje koszt $$c(T)=\sum_mc(m,T(m))$$

* (wiki) mamy $n$ książek o równej szerokości na jednej półce
  * chcemy przesunąć wszystkie książki w prawo o grubość jednej
    1. przesunąć każdą książkę o w prawo
    2. przenieść książkę po lewej na sam koniec
  * jeśli koszt jest euklidesowski $c(x,y)=\mid x-y\mid$, to oba rozwiązania mają równy koszt
  * jeśli koszt jest wypukły, np. $c(x,y)=(x-y)^2$, to pierwsze rozwiązanie jest tańsze
* rozwiązanie optymalnego transportu OT może służyć jako sposób znalezienia odległości między rozkładami

## Sformułowanie rozwiązania
* niech $c(x,y):X\times X\longrightarrow\mathbb{R}_+$ będzie funkcją kosztu
* $P(X\sim P_X, Y\sim P_G)$ zbiór rozkładów z rozkładami brzegowymi $P_X$ i $P_G$
* wtedy $c(x,y)=d^p(x,y)$ dla $p\geq1$ jest __odległością Wassersteina__ rzędu $p$
* dla $c(x,y)=d(x,y)$ zachodzi $$W_1(P_X,P_G)=\sup_{f\in\mathcal{F}_L}\mathbb{E}_{X\sim{}P_X}[f(X)]-\mathbb{E}_{Y\sim{}P_G}[f()]$$

## Zastosowanie do modeli generatywnych
1. samplujemy kod $z\sim p(z)$, gdzie $p(z)$ jest założonym priorem
2. $z$ jest mapowane do obrazu wyjściowego przez generator
3. z tego wynika gęstość $$p_G(x)=\int_Z p_G(x\mid z)p_Z(z)dz$$
  * zakładamy tu, że dekoder jest deterministyczny $x=G(z)$


Koszt OT jest planem transportu __przez__ mapowanie $G$
  * zamiast szukać powiązania między zmiennymi losowymi w X: jednej z $P_X$, a drugiej z $P_G$
  * wystarczy znaleźć takie $Q(z\mid x)$ takiej, że 
  $$Q_Z(z)=\mathbb{E}_{X\sim{}P_X}[Q(z\mid x)]$$jest identyczne z rokładem prior $P_Z$
  

To pozwala na zdefiniowanie kosztu $D_{WAE}$
$$D_{WAE}(P_X,P_G)=\inf_{Q(z\mid x)}\mathbb{E}_{P_X}\mathbb{E}_{Q(z\mid x)}[c(x,G(z))]+\lambda D_Z(Q_Z,P_Z),$$
gdzie $D_Z()$ jest dywergencją między $P_Z$ a $Q_Z$, $\lambda>0$
<img src="../nn_figures/WAE-VAE.pdf" width="80%">

(Tolstikhin,Busquet,Gelly,Scholkopf)
1. VAE (po lewej) i WAE (po prawej) minimalizują sumę kosztu rekonstrukcji oraz kosztu niezgodności między priorem $P_Z$ a rozkładem indukowanym przez enkoder $Q$
2. VAE wymusza by $Q(z\mid x)$ odpowiadało $P_Z$ dla wszystkich $x\sim P_X$ (po lewej)
  * każda kula (czerwona) ma odpowiadać $P_Z$ (biała kula)
  * kule czerwone przecinają się, co może powodować problemy z rekonstrukcją
3. WAE wymusza miksturę $Q_Z=\int{}Q(z\mid x)p_X(x)dx$ (zielona) by pasowało do $P_Z$
  * kody dla wielu przykładów mają większą szansę nie nakładania się
  * co daje lepszą rekonstrukcję

## WAE-GAN
* $D_Z(Q_Z,P_Z)=D_{JS}(Q_Z,P_Z)$
* enkoder $Q$, dekoder $G$, dyskryminator $D$
* w pętli aż do zbieżności
  1. wylosowanie próbki $\{x_1,…,x_n\}$ ze zbioru uczącego
  2. wylosowanie próbki $\{z_1,…,z_n\}$ z $P_Z$
  3. samplowanie $\tilde{z}_i\sim Q(z\mid x_i)$ dla $i=1,…,n$
  4. poprawa dyskryminatora przez __wzrost__ $$\frac{\lambda}{n}\sum\log\,D(z_i)+\log\,(1-D(z_i))$$
  5. poprawa enkodera $Q$ i dekodera $G$ przez minimalizację $$\frac{1}{n}\sum{}c(x_i,G(\tilde{x_i}))-\log\,D(\tilde{z}_i)$$
  

## WAE-MMD
* $\lambda>0$, kernel charakterystyczny $k$
* inicjalizacja enkodera $Q$, dekodera $G$
* w pętli aż do zbieżności parametrów
  1. wylosowanie próbki $\{x_1,…,x_n\}$ ze zbioru uczącego
  2. wylosowanie próbki $\{z_1,…,z_n\}$ z $P_Z$
  3. samplowanie $\tilde{z}_i\sim Q(z\mid x_i)$ dla $i=1,…,n$
  5. poprawa enkodera $Q$ i dekodera $G$ przez minimalizację $$\frac{1}{n}\sum{}c(x_i,G(\tilde{x_i}))+\frac{\lambda}{n(n-1)}\sum_{j\neq{}l}k(z_l,z_j)+\frac{\lambda}{n(n-1)}\sum_{j\neq{}l}k(\tilde{z}_l,\tilde{z}_j)-\frac{2\lambda}{n^2}\sum_{j,l}k(z_l,\tilde{z}_j)$$



### Warunki 
* dla poprawności funkcje obliczane w rozwiązaniu OT muszą być lipschitzowskie $$\mid(f(x_1)-f(x_2)\mid\leq\,L\mid{}x_1-x_2\mid$$
* aby to zapewnić w sieciach neuronowych potrzebnych jest wiele tricków
  * clipowanie wag dyskryminatora po wykonaniu update-u
  * lepiej sprawdza się nawet clipowanie gradientów
  * w związku z tym dyskryminator musi zwykle wykonać szereg cyklów na jedno poprawienie generatora
* w modelu MMD zwykle konieczne jest próbkowanie z rozkładu prior
  * wygodnie jest zastąpić odpowiednim kernelem, by można było mierzyć odległość między próbką z prioru a próbką z $Q(z\mid x)$ w sposób analityczny
  * takim kernelem może być kernel Gausowski
* kernel gausowski bardzo szybko spada
  * odległosci dla przykładów w ogonach są zupełnie nieznaczące liczbowo
  * potrzebny jest kernel z grubymi końcami, ale możliwy do obliczeń analitycznych